In [1]:
%AddJar -magic https://brunelvis.org/jar/spark-kernel-brunel-all-2.3.jar -f

Starting download from https://brunelvis.org/jar/spark-kernel-brunel-all-2.3.jar
Finished download of spark-kernel-brunel-all-2.3.jar


## Churn Rate Trend

In [2]:
//Load data from DB2 for z/OS using JDBC driver
//Load data from DB2 for z/OS using JDBC driver
val churnDataRate = spark.read.format("jdbc").
                        options(Map("driver" -> "com.ibm.db2.jcc.DB2Driver",
                         "url" -> "jdbc:db2://9.125.72.72:430/LOCDB11", 
                         "user" -> "tuser01","password" -> "c4deshop", 
                         "dbtable" -> "SA.CHURN_RATE")).load()

churnDataRate.show(5)

+----+-------+------------+----------+
|YEAR|QUARTER|QUARTER_YEAR|CHURN_RATE|
+----+-------+------------+----------+
|2014|      1|      1Q14  |      18.1|
|2014|      2|      2Q14  |      18.7|
|2014|      3|      3Q14  |      19.3|
|2014|      4|      4Q14  |      19.9|
|2015|      1|      1Q15  |      20.5|
+----+-------+------------+----------+
only showing top 5 rows



In [3]:
%%brunel data('churnDataRate') x(QUARTER_YEAR) y(CHURN_RATE) bar tooltip(#all) sort(YEAR:ascending, QUARTER:ascending) tooltip(#all) axes(x:'Time', y:'Churn Rate':grid) 
:: width=800, height=500

In [4]:
import org.apache.spark.sql.SparkSession

//Load data from DB2 for z/OS using JDBC driver
val churnData = spark.read.format("jdbc").
                        options(Map("driver" -> "com.ibm.db2.jcc.DB2Driver",
                         "url" -> "jdbc:db2://9.125.72.72:430/LOCDB11", 
                         "user" -> "tuser01","password" -> "c4deshop", 
                         "dbtable" -> "SA.CUST_SUM")).load()

churnData.show(5)

+----------+---+---+---------+----------+-----------+--------+-----+--------+------------------+-------+-----------------+---------+-----+----------------+---------+-----------+
|   CUST_ID|SEX|AGE|EDUCATION|INVESTMENT|     INCOME|ACTIVITY|CHURN|YRLY_AMT|      AVG_DAILY_TX|YRLY_TX|       AVG_TX_AMT|NEGTWEETS|STATE| EDUCATION_GROUP|TwitterID|CHURN_LABEL|
+----------+---+---+---------+----------+-----------+--------+-----+--------+------------------+-------+-----------------+---------+-----+----------------+---------+-----------+
|1009530860|  F| 84|        2|114368.000|3852862.000|       5|    0|700259.0|0.9178079962730408|    335|2090.320068359375|        3|   TX|Bachelors degree|        0|      false|
|1009544000|  F| 44|        2| 90298.000|3849843.000|       1|    0|726977.0|0.9506850242614746|    347|  2095.0400390625|        2|   CA|Bachelors degree|        0|      false|
|1009534260|  F| 23|        2| 94881.000|3217364.000|       1|    1|579084.0|0.9205480217933655|    336|  1723

In [5]:
import org.apache.spark.sql.functions._ 

val groupedByState = churnData.groupBy("STATE").
                        agg(avg("INCOME") as "mean_income")
groupedByState.show(5)


+-----+-------------+
|STATE|  mean_income|
+-----+-------------+
|   SC|32230.0000000|
|   AZ|25862.1609195|
|   LA|26734.8831169|
|   MN|25064.2842105|
|   NJ|31334.7979798|
+-----+-------------+
only showing top 5 rows



## Income by state

In [6]:
%%brunel data('groupedByState') map key(STATE) x(STATE) color(mean_income) label(STATE) tooltip(#all) :: width=800, height=500

## Distribution by churn

In [7]:
%%brunel data('churnData') 
         x(AGE) y(#count:linear) color(CHURN_LABEL) bin(AGE) interaction(select) stack bar tooltip(#all) filter(CHURN_LABEL) legends(none) |
         x(AVG_DAILY_TX) y(#count:linear) color(CHURN_LABEL) opacity(#selection) bin(AVG_DAILY_TX) stack bar tooltip(#all) axes(x:10:'AVG DAILY TX', y)|
         x(AVG_TX_AMT) y(#count:linear) color(CHURN_LABEL) opacity(#selection) bin(AVG_TX_AMT) stack bar tooltip(#all)  axes(x,y) legends(none) |
        x(INCOME) y(#count:linear) color(CHURN_LABEL) opacity(#selection) bin(INCOME) stack bar tooltip(#all) tooltip(#all) axes(x,y) legends(none)
:: width=800, height=600

In [8]:
%%brunel data('churnData')
         x(SEX) y(#count:linear) color(CHURN_LABEL) stack bar tooltip(#all) sort(SEX) interaction(select) filter(CHURN_LABEL) axes(x:'GENDER', y) legends(none) |
         x(ACTIVITY) y(#count:linear) color(CHURN_LABEL) stack bar tooltip(#all) sort(ACTIVITY) opacity(#selection)  | 
         x(EDUCATION_GROUP) y(#count:linear) color(CHURN_LABEL) stack bar tooltip(#all) sort(#count) opacity(#selection)  axes(x:'',y) legends(none)
:: width=900, height=600

In [11]:
import org.apache.spark.sql.functions._ 

val churnGroupedByState = churnData.groupBy("STATE").
                        agg(avg("CHURN") as "mean_churn")
churnGroupedByState.show(5)


+-----+----------+
|STATE|mean_churn|
+-----+----------+
|   SC|       0.0|
|   AZ|       0.0|
|   LA|       0.0|
|   MN|       0.0|
|   NJ|       0.0|
+-----+----------+
only showing top 5 rows



In [12]:
%%brunel data('churnGroupedByState') map key(STATE) x(STATE) color(mean_churn) label(STATE) tooltip(#all) :: width=800, height=500